ChEn-5310: Computational Continuum Transport Phenomena Spring 2021 UMass Lowell; Prof. V. F. de Almeida **25Jan21**

# 01. Introduction

---

## Table of Contents<a id="toc"></a>
* [Objectives](#obj)
---

## [Objectives](#toc)<a id="obj"></a>

 + Introductory comments on variational forms and basis functions.
 + Elementary use of basis functions in approximation with least-squares method.
 + Elementary description of a least-squares Galerkin spectral element method.
 + [Handwritten notes in OneNote](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/EiT9S8dWWQ5LuvalDfeDuOoBs-_tgLPFR83xU6WumdI9xg?e=fQXd40).

## [Least-Squares Residual Minimization w/ Fourier Basis Functions](#toc)<a id="obj"></a>
